# Closures and Free Variables
- closure is as a function plus an extended scope that contains a free Variables.
- The free variable's value in the object/cell points to-so that could change over time.
- Every time the function in the closure is called and the free variable is referenced, python looks up the cell object, and then whatever the cell is pointing to..

In [72]:
# hoc
def outer(word):


    times = 8
    def inner():
        print(word,times,'times')
        # times refers to the outer scope. the non-local variable times is called free variable.


    return inner # when we consider inner_func , we really look at: inner_func and the free variable word. they have to be bound together, called closure.


f = outer('hello')
# when we return inner, we are actually returning the closure.
f() # when we call f(), python determined the value of x in the extended scope.
# But, outer had finished running before we called f(), it's scope was gone. that because of scope.


hello 8 times


## Python cells and multi-scoped variables.

In [73]:
def outer():
    x = 'python' # here the value of x is shared between two scopes -> outer and closure.
    def inner():
        print(x)
    return inner # the label x is in two different scopes, but always reference the same 'value'

fn = outer()
fn()
# python does this by creating a cell as an intermediary object/cell.

python


- a cell in memory address point to another memory containing str 'python'
- the cell is pointed by variable x from outer function as well as pointed by variable x from inner function.
- There is nothing new, this is same as non-local variables.
##
- in effect, both variables x (in outer and inner), point to the same cell.
- When requesting the value of the variable, python will, "double-hop" to get the final value.
- When we call the returned function from closure. it get track or remember the cell. which is not yet garbage collected.

![](files/closure.png)

## introspection

In [74]:
def outer():
    a = 100
    x = 'python'
    def inner():
        a = 10 #local variable
        print(f'{x} rocks!')
    return inner

fn = outer()
fn()

python rocks!


In [75]:
fn.__code__.co_freevars

('x',)

In [76]:
fn.__closure__ # cell at address point to str object to another address.

(<cell at 0x000001F6370EC610: str object at 0x000001F6333EE4B0>,)

## Modifying free variables.

In [77]:
def counter():


    count = 0
    def inc():
        nonlocal count
        count +=1
        return count
    

    return inc


fn = counter()
print(fn()) # count's reference changed from the object 0 to the object 1
fn()

1


2

## Multiple instances of closures
- every time a function is called, a new scope is created.
- same as, if that function generates a closures, a new closures is created every time as well.

In [78]:
def counter():


    count = 0
    def inc():
        nonlocal count
        count +=1
        return count
    

    return inc


fn1 = counter()
fn2 = counter()
# fn1()
print(fn1.__closure__)
print(fn2.__closure__)

(<cell at 0x000001F636B3F4C0: int object at 0x000001F630C56910>,)
(<cell at 0x000001F636B224F0: int object at 0x000001F630C56910>,)


In [79]:
# The cells are different.

## Shared Extended Scopes

In [80]:
def outer():
    count = 0

    def inc1():
        nonlocal count # count is free variable-bound to count in the extended scope.
        count +=1
        return count
    
    def inc2():
        nonlocal count # count is free variable-bound to the same count.
        count +=1
        return count
    
    return inc1,inc2

f1,f2 = outer()

print(f1())
print(f1())
print(f2())

print(f1.__closure__)
print(f2.__closure__)

1
2
3
(<cell at 0x000001F636EECA60: int object at 0x000001F630C56970>,)
(<cell at 0x000001F636EECA60: int object at 0x000001F630C56970>,)


In [81]:
# we might think shared extended scope is hightly unusual, but it's not.

def adder(n):
    def inner(x):
        return x + n
    
    return inner

add_1 = adder(1) # three diff closures, no shared scopes.
add_2 = adder(2)
add_3 = adder(3)

print(add_1(10))
print(add_2(10))
print(add_3(10))

11
12
13


In [82]:
# a function is a function.
# a function become closure if it contain free-variables.
# lamda is not-necessary to be a closure and vice-versa.
# just closure have a extended scope.
# eg:
adders = []
for n in range(1,4):
    adders.append(lambda x:x+n) # n is free variables so, it becomes now closures.
    # n =1: the free variable in the lamda is n, add it is bound to the n we created in the loop.
    # n =2: the free variable in the lamda is n, add it is bound to the (same) n we created in the loop.
    # n =3: the free variable in the lamda is n, add it is bound to the (same) n we created in the loop.
    # point/bound to the same cell, and the cell point to the n.

print(adders[0](10)) # the n is evaluated when adders[0] is called.
print(adders[1](10))
print(adders[2](10))

13
13
13


## Nested Closure

In [83]:
def increment(n):
    # inner and n is closure
    def inner(start):
        current = start
        # inc , current , n is a closure

        def inc():
            nonlocal current # free variable.
            current +=n # non local so, free variable.
            return current
        return inc
    return inner

fn = increment(2) # inner
print(fn.__code__.co_freevars)

inc_2 = fn(100) # inc
print(inc_2.__code__.co_freevars)

inc_2()

('n',)
('current', 'n')


102

## Closure Application 1

In [84]:
# normal class
class Avrager:
    def __init__(self):
        self.number = []
    
    def add(self,number):
        self.number.append(number)
        total = sum(self.number) # every time we have to calculate its not efficient
        count = len(self.number)
        return total/count

In [85]:
a = Avrager()
a.add(10)

10.0

In [86]:
a.add(50)

30.0

In [87]:
def averager():
    numbers =[]
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total/count
    return add

In [88]:
a = averager()
a(10)

10.0

In [89]:
a(20)

15.0

In [90]:
b = averager()
b(10)

10.0

In [91]:
print(a.__closure__)
print(b.__closure__)

(<cell at 0x000001F636EEC6D0: list object at 0x000001F636F9CAC0>,)
(<cell at 0x000001F636EEC250: list object at 0x000001F636FBC440>,)


In [92]:
# fix using closure
def averager():
    total,count = 0,0
    def add(number):
        nonlocal total,count
        total += number
        count +=1
        return total/count
    return add

# fixing in class
class Averager:
    def __init__(self):
        self.total,self.count = 0,0
    
    def add(self,number):
        self.total += number
        self.count +=1
        return self.total/self.count

# these two are same.

In [93]:
a = averager()
a.__closure__

(<cell at 0x000001F637120D90: int object at 0x000001F630C56910>,
 <cell at 0x000001F637120310: int object at 0x000001F630C56910>)

In [94]:
a(10)
a(20)

15.0

In [95]:
from time import perf_counter

In [96]:
print(perf_counter())
print(perf_counter())

3468.033787
3468.0344829


In [97]:
class Timer:
    def __init__(self):
        self.start = perf_counter()

    def __call__(self):
        return perf_counter()-self.start
    
def TIMER():
    start = perf_counter()
    def poll():
        return perf_counter()-start
    return poll
#both are same.

In [98]:
t1 = Timer()

In [99]:
t1()

0.022681899999952293

In [100]:
t1()

0.054467899999963265

In [101]:
t2 = TIMER()
t2()

8.969999998953426e-05

## NOTE
- In general if you have classes that are preety straight forward there's only one call you are intrested in. Even if you have more thaan one value being stored. We can rewrite it as a closure and often times the code will be simple.

## Closure Application 2
- Adding functionality to simple function

In [102]:
# This is the much easier nested closure.

def counter(initial_value=0):
    def inc(increment=1):
        nonlocal initial_value
        initial_value +=increment
        return initial_value
    return inc # inc is closure

counter1 = counter(0)
counter1()

1

In [103]:
counter1()

2

In [104]:
# hoc -> taking function as a paremeter.
def counter(fn,counters):
    cnt = 0

    def inner(*args,**kwargs):
        nonlocal cnt
        cnt +=1
        counters[fn.__name__] = cnt # no need to specify non-local counter.
        return fn(*args,**kwargs)
    return inner

def add(a,b):
    return a+b

def mul(a,b):
    return a*b

In [105]:
c = dict()
add = counter(add,c)
add.__closure__

(<cell at 0x000001F6370A8D60: int object at 0x000001F630C56910>,
 <cell at 0x000001F6370A80A0: dict object at 0x000001F636A81640>,
 <cell at 0x000001F6370A8FD0: function object at 0x000001F636AE6700>)

In [106]:
counter_add.__code__.co_freevars

('cnt', 'counters', 'fn')

In [107]:
add(10,20)
add(10,20)

30

In [108]:
mul = counter(mul,c)
mul(2,5)
mul(2,5)
mul(2,5)

10

In [109]:
c

{'add': 2, 'mul': 3}

In [110]:
# using closure to find factorial
def fact(n):
    product = 1
    for i in range(2,n+1):
        product *=i
    return product

In [111]:
fact = counter(fact,c)
fact(3)
fact(3)
fact(2)
fact(4)

24

In [112]:
c

{'add': 2, 'mul': 3, 'fact': 4}

In [113]:
fact.__closure__

(<cell at 0x000001F6370A8BE0: int object at 0x000001F630C56990>,
 <cell at 0x000001F6370A8C10: dict object at 0x000001F636A81640>,
 <cell at 0x000001F6370A8F10: function object at 0x000001F636AE6160>)